In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/291_Project/

In [ ]:
!pip install tensorflow_addons
!pip install transformers

In [ ]:
import pandas as pd
import torch
import numpy as np
import tensorflow_addons as tfa
import tensorflow as tf
#deduplicate a csv
import itertools
import string
import re
import ast
import sys
import tqdm
from transformers import AutoModel, AutoTokenizer

In [ ]:
model = tf.keras.models.load_model('my_model')
tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")
simcse_pretrained_model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")

device = "cuda" if torch.cuda.is_available() else "cpu"
simcse_pretrained_model = simcse_pretrained_model.to(device)

In [ ]:
def get_embeddings(text1):
  #text1 should be list
  inputs1 = tokenizer(list(text1), padding=True, truncation=True, return_tensors="pt").to(device)
  with torch.no_grad():
      embed = simcse_pretrained_model(**inputs1, output_hidden_states=True, return_dict=True).pooler_output.detach().cpu().numpy()
  return embed



def generate_predictions(unique_col_vals, embeddings_dict):
  pairs =  [(unique_col_vals[i],unique_col_vals[j]) for i in range(len(unique_col_vals)) for j in range(i+1, len(unique_col_vals))]

  embeddings1 = []
  embeddings2 = []
  predictions = []

  for pair in pairs:
    embeddings1.append(embeddings_dict[pair[0]])
    embeddings2.append(embeddings_dict[pair[1]])

  batch_size = 50000
  idx = 0
  while idx < len(embeddings1):
    start = idx
    end = idx+batch_size
    end = len(embeddings1) if end > len(embeddings1) else end
    t1 = np.array(embeddings1[start:end])
    t2 = np.array(embeddings2[start:end])
    y_predicted = list(model.predict(x=[t1,t2]))
    predictions = predictions + y_predicted
    idx += batch_size


  output = {}
  for idx,pair in enumerate(pairs):
    output[pair] = predictions[idx][0]

  return output


def preprocess(text):
  clean_text = text.strip()
  clean_text = clean_text.replace(' ','')
  clean_text = clean_text.lower()
  clean_text = re.sub(r'\d+','',clean_text)
  lookup_table = clean_text.maketrans('', '', string.punctuation)
  clean_text = clean_text.translate(lookup_table)
  return clean_text

def get_matches(col_val, col_unique_set, predictions):
  preds = []
  for idx, uni_val in enumerate(col_unique_set):
    if (col_val,uni_val) in predictions.keys():
      preds.append(predictions[(col_val,uni_val)])
    elif (uni_val,col_val) in predictions.keys():
      preds.append(predictions[(uni_val,col_val)])
    else:
      print("key missing, some issue",col_val,uni_val)
      preds.append(-1)
  return preds


In [ ]:
midwest_df = pd.read_csv("downstream-data 2/Midwest_Survey/Midwest_Survey.csv")
midwest_df = midwest_df.dropna()
categorical_cols = ["In your own words, what would you call the part of the country you live in now?"]
column = categorical_cols
for col in column:
  midwest_df[col] = midwest_df[col].map(preprocess)
# for column in categorical_cols:
unique_col_vals = midwest_df[column].values.flatten().tolist()
unique_col_vals = list(set(unique_col_vals))

embeddings = get_embeddings(unique_col_vals)

embeddings_dict = {}
for idx, val in enumerate(unique_col_vals):
  embeddings_dict[val] = embeddings[idx]

predictions = generate_predictions(unique_col_vals, embeddings_dict)

1396/1396 [==============================] - 5s 3ms/step


In [ ]:
unique_set = set([unique_col_vals[0]]) # Add the 0th element
deduplicated_value = {unique_col_vals[0]: unique_col_vals[0]}

a = 0
b = 0
c = 0
d = 0
for col_val in unique_col_vals[1:]:
  unique_set = list(unique_set)
  preds = get_matches(col_val, unique_set, predictions)
  preds_rounded = np.round(np.array(preds)).tolist()
  #TODO: Check if preds contains -1, in that case do something

  if sum(preds_rounded) == 0: #no matches, therefore no deduplication, this is a new entity
    a+=1
    unique_set.append(col_val)
    deduplicated_value[col_val] = col_val
  elif sum(preds_rounded) == 1: #exactly one match, ideal case
    b+=1
    uni_val = unique_set[preds_rounded.index(1)]
    uni_freq = len(midwest_df[midwest_df[categorical_cols[0]]==uni_val])
    col_freq = len(midwest_df[midwest_df[categorical_cols[0]]==col_val])
    best_val = uni_val if uni_freq > col_freq else col_val
    unique_set.append(best_val)
    deduplicated_value[col_val] = best_val
  elif sum(preds_rounded) > 1: #multiple matches, due to false positives, need to find the best match -> use confidence

    c+=1
    preds_np = np.array(preds)
    best_val = unique_set[preds_np.argmax()]
    unique_set.append(best_val)
    deduplicated_value[col_val] = best_val
  else:
    d+=1
    unique_set.append(col_val)
    deduplicated_value[col_val] = col_val

  unique_set = set(unique_set)


In [ ]:
len(midwest_df[midwest_df[categorical_cols[0]]=="california"])

11

In [ ]:
midwest_df[categorical_cols[0]] = midwest_df[categorical_cols[0]].apply(lambda x: deduplicated_value[x])
midwest_df.to_csv("downstream-data 2/Midwest_Survey/Midwest_Survey_dedup2.csv")

In [ ]:
a,b,c,d

(80, 16, 603, 0)